# Eksempel på agent til booking af hotel og fly

Denne løsning vil hjælpe dig med at booke flybilletter og hotel. Scenariet er en rejse fra London Heathrow LHR den 20. februar 2024 til New York JFK med retur den 27. februar 2025, hvor der flyves på økonomiklasse med British Airways. Jeg ønsker at bo på et Hilton-hotel i New York, så venligst oplys om omkostningerne for fly og hotel.


# Initialiser Azure AI Agent Service og få konfigurationsoplysninger fra **.env**

### **.env** 

Opret en .env-fil 

**.env** indeholder forbindelsesstrengen til Azure AI Agent Service, modellen brugt af AOAI, samt den tilsvarende Google API Search service API, ENDPOINT osv.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Navnet på din Azure AI Agent Service Model Deployment"

[**NOTE**] Du skal bruge en model med 100.000 Rate Limit (Tokens per minut) og en Rate Limit på 600 (Forespørgsler per minut)

  Du kan få modellen i Azure AI Foundry - Model og Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Forbindelsesstrengen til dit Azure AI Agent Service-projekt"

  Du kan finde projektets forbindelsesstreng i projektoversigten på AI ​​Foundry Portal-skærmen.

- **SERPAPI_SEARCH_API_KEY** = "Din SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Din SERPAPI Search Endpoint"

For at få Model Deployment Name og Project Connection String til Azure AI Agent Service skal du oprette Azure AI Agent Service. Det anbefales at bruge [denne skabelon](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) til at oprette den direkte （***Bemærk:*** Azure AI Agent Service er i øjeblikket begrænset til visse regioner. Det anbefales, at du henviser til [dette link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) for at indstille regionen)

Agenten skal have adgang til SERPAPI. Det anbefales at registrere dig via [dette link](https://serpapi.com/searches). Efter registrering kan du få en unik API KEY og ENDPOINT


# Opsætning

For at køre denne notebook skal du sikre dig, at du har installeret de nødvendige biblioteker ved at køre `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Din Semantic Kernel-version bør være mindst 1.27.2.


Indlæs dine .env-filindstillinger og ressourcer. Sørg venligst for, at du har tilføjet dine nøgler og indstillinger og oprettet en lokal .env-fil.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Log ind på Azure

Du skal nu logge ind på Azure. Åbn en terminal, og kør følgende kommando:

```bash
az login
```

Denne kommando vil bede dig om at indtaste dine Azure-legitimationsoplysninger, så Azure AI Agent-tjenesten kan fungere korrekt.


# Forklaring:
Dette er en variabel, der gemmer API-nøglen til adgang til en SERP (Search Engine Results Page) API-tjeneste. En API-nøgle er en unik identifikator, der bruges til at autentificere forespørgsler, der er knyttet til din konto.

Formål: Formålet med denne linje er at gemme API-nøglen i en variabel, så den kan bruges til at autentificere forespørgsler til SERP API-tjenesten. API-nøglen er nødvendig for at få adgang til tjenesten og udføre søgninger.
Sådan får du en SERP API-nøgle: For at få en SERP API-nøgle skal du følge disse generelle trin på https://serpapi.com (de præcise trin kan variere afhængigt af den specifikke SERP API-tjeneste, du bruger):

Vælg en SERP API-tjeneste: Der findes flere SERP API-tjenester, såsom SerpAPI, Google Custom Search JSON API og andre. Vælg den, der bedst opfylder dine behov.

Opret en konto: Gå til hjemmesiden for den valgte SERP API-tjeneste og opret en konto. Du skal muligvis angive nogle grundlæggende oplysninger og bekræfte din e-mailadresse.

Opret en API-nøgle: Efter tilmelding skal du logge ind på din konto og navigere til API-sektionen eller dashboardet. Find en mulighed for at oprette eller generere en ny API-nøgle.
Kopier API-nøglen til din .env-fil.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Forklaring:
BASE_URL: Dette er en variabel, der gemmer basis-URL'en for SERP API-endepunktet. Variabelnavnet BASE_URL er en konvention, der angiver, at denne URL er udgangspunktet for at lave API-forespørgsler.  
'https://serpapi.com/search':  

Dette er den faktiske URL-streng, der er tildelt BASE_URL-variablen. Den repræsenterer endepunktet til at udføre søgeforespørgsler ved hjælp af SERP API.

# Formål:
Formålet med denne linje er at definere en konstant, der indeholder basis-URL'en for SERP API. Denne URL vil blive brugt som udgangspunkt for at opbygge API-forespørgsler til at udføre søgeoperationer.

# Anvendelse:
Ved at definere basis-URL'en i en variabel kan du nemt genbruge den i hele din kode, når du har brug for at lave forespørgsler til SERP API. Dette gør din kode mere vedligeholdelsesvenlig og reducerer risikoen for fejl ved at hardkode URL'en flere steder. Det aktuelle eksempel er https://serpapi.com/search?engine=bing, som bruger Bing-søge-API'en. Forskellige API'er kan vælges på https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Forklaring:

Her er, hvor din plugin-kode er placeret.

Klasse Definition: `class BookingPlugin`: Definerer en klasse ved navn BookingPlugin, som indeholder metoder til booking af hoteller og fly.

Hotel Booking Metode:

- `@kernel_function(description="booking hotel")`: En dekorator, der beskriver funktionen som en kernel-funktion til booking af hoteller.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definerer en metode til booking af hoteller med annoterede parametre og returtype.

Metoden konstruerer en ordbog med parametre til hotelbooking-forespørgslen og sender en GET-forespørgsel til SERP API'en. Den tjekker svarstatus og returnerer hoteloplysningerne, hvis det lykkes, eller None, hvis forespørgslen mislykkedes.

Fly Booking Metode:

- `@kernel_function(description="booking flight")`: En dekorator, der beskriver funktionen som en kernel-funktion til booking af fly.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definerer en metode til booking af fly med annoterede parametre og returtype.

Metoden konstruerer ordbøger med parametre til forespørgsler om udgående og returfly og sender GET-forespørgsler til SERP API'en. Den tjekker svarstatus og tilføjer flyoplysningerne til resultatstrengen, hvis det lykkes, eller udskriver en fejlmeddelelse, hvis forespørgslen mislykkedes. Metoden returnerer resultatstrengen, der indeholder flyoplysningerne.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Forklaring:
Importudsagn: Importer nødvendige moduler til Azure-legitimationsoplysninger, AI-agent, chatbeskedindhold, forfatterrolle og kernel-funktionsdekoration.

Asynkron kontekststyring: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Dette opsætter en asynkron kontekststyring til at håndtere Azure-legitimationsoplysninger og oprette en AI-agentklient.

Agentnavn og instruktioner:
- `AGENT_NAME = "BookingAgent"`: Definerer agentens navn.
- `AGENT_INSTRUCTIONS = """..."""`: Giver detaljerede instruktioner til agenten om, hvordan den skal håndtere bookingforespørgsler.

Opret agentdefinition: `agent_definition = await client.agents.create_agent(...)`: Opretter en agentdefinition med den specificerede model, navn og instruktioner.

Opret AzureAI-agent: `agent = AzureAIAgent(...)`: Opretter en AzureAI-agent ved hjælp af klienten, agentdefinitionen og det definerede plugin.

Opret tråd: `thread: AzureAIAgentThread | None = None`: Opretter en tråd til agenten. Det er ikke nødvendigt først at oprette en tråd - hvis værdien `None` angives, vil en ny tråd blive oprettet under den første kaldelse og returneret som en del af svaret.

Brugerinput: `user_inputs = ["..."]`: Definerer en liste over brugerinput, som agenten skal behandle.

I finally-blokken slettes tråden og agenten for at frigøre ressourcer.


# Godkendelse

Klassen `DefaultAzureCredential` er en del af Azure SDK for Python. Den tilbyder en standardmetode til at godkende med Azure-tjenester. Den forsøger at godkende ved hjælp af flere metoder i en bestemt rækkefølge, såsom miljøvariabler, administreret identitet og Azure CLI-legitimationsoplysninger.

Asynkrone operationer: aio-modulet angiver, at klassen DefaultAzureCredential understøtter asynkrone operationer. Det betyder, at du kan bruge den med asyncio til at udføre ikke-blokerende godkendelsesanmodninger.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
